# Pick and place sequence

This example notebook demonstrates how to create a sequential pick-and-place process.

In particular, this example covers:

- Behavior tree nodes
    - `Task`
    - `Sequence`
    - `SubTree`

<div class="alert alert-info">

**Important**

This notebook requires a running Flowstate solution to connect to. To start a solution:

1. Navigate to [portal.intrinsic.ai](https://portal.intrinsic.ai/) and sign in
   using your registered Flowstate account.

1. Do **one** of the following:
    - Create a new solution:
        1. Click "Create new solution" and choose "From an example".
        1. Select `pick_and_place:pick_and_place_module2`
        1. Click "Create".
    - Or open an existing solution that was created from the `pick_and_place:pick_and_place_module2` example:
        1. Hover over the solution in the list.
        1. Click "Open solution" or "Start solution".

1. Recommended: Keep the browser tab with the Flowstate solution editor open to watch the effect of notebook actions such as running a skill. You can simultaneously interact with the solution through the web UI and the notebook.

</div>

First, connect and define convenience shortcuts:

In [ ]:
from intrinsic.math.python import data_types
from intrinsic.solutions import deployments
from intrinsic.solutions import behavior_tree as bt

solution = deployments.connect_to_selected_solution()

executive = solution.executive
resources = solution.resources
skills = solution.skills
world = solution.world
simulator = solution.simulator

attach_object_to_robot = skills.ai.intrinsic.attach_object_to_robot
control_suction_gripper = skills.ai.intrinsic.control_suction_gripper
detach_object = skills.ai.intrinsic.detach_object
enable_gripper = skills.ai.intrinsic.enable_gripper
move_robot = skills.ai.intrinsic.move_robot
plan_grasp = skills.ai.intrinsic.plan_grasp

Create the required skills:

In [ ]:
collisions_disabled = move_robot.intrinsic_proto.world.CollisionSettings(
    disable_collision_checking=True
)

init_gripper = enable_gripper(
    clear_faults=True, gripper=resources.picobot_gripper)

move_home = move_robot(
    motion_segments=[
        move_robot.intrinsic_proto.skills.MotionSegment(
            joint_position=world.robot.joint_configurations.home,
            motion_type=move_robot.intrinsic_proto.skills.MotionSegment.MotionType.ANY,
        )
    ]
)

# Moves the global frames `world.grasp_frame` and `world.pregrasp_frame` to
# suitable grasp/pregrasp positions above the building block.
plan_block_grasp = plan_grasp(
    objects=[world.building_block0],
    tool_frame=world.picobot_gripper.tool_frame,
    advanced_params=(
        plan_grasp.intrinsic_proto.manipulation.skills.PlanGraspAdvancedParams(
            product_part_name="building_block",
            output_pregrasp_frame=world.pregrasp_frame,
            output_grasp_frame=world.grasp_frame,
        )
    ),
)

move_pregrasp = move_robot(
    motion_segments=[
        move_robot.intrinsic_proto.skills.MotionSegment(
            cartesian_pose=move_robot.intrinsic_proto.world.geometric_constraints.PoseEquality(
                moving_frame=world.picobot_gripper.tool_frame,
                target_frame=world.pregrasp_frame,
            ),
            motion_type=move_robot.intrinsic_proto.skills.MotionSegment.MotionType.ANY,
        )
    ]
)

# Move to grasp pose: Disable collision checking because at the end of the move
# the gripper tip will touch the building block.
move_grasp_unsafe = move_robot(
    motion_segments=[
        move_robot.intrinsic_proto.skills.MotionSegment(
            cartesian_pose=move_robot.intrinsic_proto.world.geometric_constraints.PoseEquality(
                moving_frame=world.picobot_gripper.tool_frame,
                target_frame=world.grasp_frame,
            ),
            motion_type=move_robot.intrinsic_proto.skills.MotionSegment.LINEAR,
            collision_settings=collisions_disabled,
        )
    ]
)

# Turn on the suction gripper which will "attach" the building block in the
# simulation/real world.
grasp = control_suction_gripper(
    grasp=control_suction_gripper.intrinsic_proto.eoat.GraspRequest()
)

# Reparent building block to the robot in the belief world so that the building block
# in the belief world will move together with the robot.
attach_block = attach_object_to_robot(
    gripper_entity=world.picobot_gripper, object_entity=world.building_block0
)

# Retract up by 3cm. Disable collision checking because we expect collisions at the beginning
# of the move.
# - Right after grasping, the attached block is in collision with the tray.
# - Right after releasing, the gripper tip is in collision with the building block.
move_up_unsafe = move_robot(
    motion_segments=[
        move_robot.intrinsic_proto.skills.MotionSegment(
            cartesian_pose=move_robot.intrinsic_proto.world.geometric_constraints.PoseEquality(
                moving_frame=world.picobot_gripper.tool_frame,
                target_frame=world.picobot_gripper.tool_frame,
                target_frame_offset=data_types.Pose3(
                    translation=[0, 0, -0.03]),
            ),
            motion_type=move_robot.intrinsic_proto.skills.MotionSegment.LINEAR,
            collision_settings=collisions_disabled,
        )
    ]
)

move_drop = move_robot(
    motion_segments=[
        move_robot.intrinsic_proto.skills.MotionSegment(
            cartesian_pose=move_robot.intrinsic_proto.world.geometric_constraints.PoseEquality(
                moving_frame=world.picobot_gripper.tool_frame,
                target_frame=world.target_right,
                target_frame_offset=data_types.Pose3(
                    rotation=data_types.Rotation3.from_euler_angles(
                        rpy_degrees=[180, 0, 90]
                    ),
                    translation=[0, 0, 0.05],
                ),
            ),
            motion_type=move_robot.intrinsic_proto.skills.MotionSegment.MotionType.ANY,
        )
    ]
)

# Turn off the suction gripper which will "drop" the building block in the
# simulation/real world.
release = control_suction_gripper(
    release=control_suction_gripper.intrinsic_proto.eoat.ReleaseRequest()
)

# Reparent building block back to the world root in the belief world.
detach = detach_object(
    gripper_entity=world.picobot_gripper, object_entity=world.building_block0
)

Then you can simply put all the skills in a `bt.Sequence` and execute them one after the other:

In [ ]:
world.reset()

sequence = bt.Sequence(
    [
        init_gripper,
        move_home,
        plan_block_grasp,
        move_pregrasp,
        move_grasp_unsafe,
        grasp,
        attach_block,
        move_up_unsafe,
        move_drop,
        release,
        detach,
        move_up_unsafe,
        move_home,
    ]
)

executive.run(sequence)

However, that makes it hard to see any structure in the visual representation of the process as it gets displayed in the Flowstate solution editor. The use of named `SubTree` and `Task` nodes makes this much cleaner. And it will also make it easy to re-use parts of the sequence, insert steps or to add failure handling or looping.

In [ ]:
world.reset()
simulator.reset()

initialize = bt.SubTree(
    name="Initialize",
    behavior_tree=bt.Sequence(
        [
            bt.Task(action=init_gripper, name="Enable gripper"),
            bt.Task(action=move_home, name="Move home"),
        ]
    ),
)

pick_object = bt.SubTree(
    name="Pick Object",
    behavior_tree=bt.Sequence(
        [
            bt.Task(action=plan_block_grasp, name="Plan grasp"),
            bt.Task(action=move_pregrasp, name="Move to pregrasp"),
            bt.Task(action=move_grasp_unsafe, name="Move to grasp"),
            bt.Task(action=grasp, name="Grasp block"),
            bt.Task(action=attach_block, name="Attach block"),
            bt.Task(action=move_up_unsafe, name="Move up after grasp"),
        ]
    ),
)

place_object = bt.SubTree(
    name="Place Object",
    behavior_tree=bt.Sequence(
        [
            bt.Task(action=move_drop, name="Move to drop"),
            bt.Task(action=release, name="Release block"),
            bt.Task(action=detach, name="Detach block"),
            bt.Task(action=move_up_unsafe, name="Move up after drop"),
            bt.Task(action=move_home, name="Move back home"),
        ]
    ),
)

pick_and_place = bt.BehaviorTree(
    name="Pick and place", root=bt.Sequence([initialize, pick_object, place_object])
)

executive.run(pick_and_place)

You can also visualize the behavior tree in-line in Jupyter. Note, however, that this visualization differs from the visualization in the Flowstate solution editor. In Jupyter, the tree structure of the behavior tree gets rendered directly (with the child nodes of a parent node arranged from left to right) whereas in the solution editor the tree gets rendered similar to a flow chart (e.g., the children of a `Sequence` node get arranged top to bottom and are connected with arrows). Despite these differences in visualization, the underlying data is the same (a behavior tree).

In [ ]:
pick_object.show()

In [ ]:
pick_and_place.show()